Import the relevant libraries and load in the csv file (be sure to upload it in the files section!).

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import f_oneway
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/MDST/UberDataset Current.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE,hour,day_of_week,...,TEMPERATURE,IS_DAY,RAIN,SNOWFALL,SHOWERS,WEATHER_CODE,WIND_SPEED_10M,WIND_GUSTS_10M,TEMP_WIND,WEATHER_STRESS
0,0,2016-11-05 19:20:00,2016-11-05 19:28:00,Business,Agnew,Agnew,2.2,NO_DATA,19,5,...,49.3,0.0,0.0,0.0,0.0,1.0,4.0,5.0,197.20,59.3
1,1,2016-11-04 22:12:00,2016-11-04 22:25:00,Business,Cory,Agnew,3.9,NO_DATA,22,4,...,41.6,0.0,0.0,0.0,0.0,0.0,6.2,13.3,257.92,61.1
2,2,2016-11-05 17:29:00,2016-11-05 17:40:00,Business,Renaissance,Agnew,2.8,NO_DATA,17,5,...,52.5,0.0,0.0,0.0,0.0,1.0,7.0,14.8,367.50,75.3
3,3,2016-11-06 16:05:00,2016-11-06 16:22:00,Business,Renaissance,Agnew,2.8,Meeting,16,6,...,43.9,1.0,0.0,0.0,0.0,0.0,16.3,34.6,715.57,94.8
4,4,2016-03-17 17:20:00,2016-03-17 18:02:00,Business,Midtown,Alief,15.5,Meal/Entertain,17,3,...,50.9,1.0,0.0,0.0,0.0,3.0,6.8,28.1,346.12,88.8


**Create a filtered dataframe based on whether you are in the Business Category or Personal Category group, and use this dataframe accordingly in the rest of the module:**




In [5]:
#business
df_business = df[df['CATEGORY'] == 'Business']
df_business.head()

,Unnamed: 0,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE,hour,day_of_week,...,TEMPERATURE,IS_DAY,RAIN,SNOWFALL,SHOWERS,WEATHER_CODE,WIND_SPEED_10M,WIND_GUSTS_10M,TEMP_WIND,WEATHER_STRESS
0,0,2016-11-05 19:20:00,2016-11-05 19:28:00,Business,Agnew,Agnew,2.2,NO_DATA,19,5,...,49.3,0.0,0.0,0.0,0.0,1.0,4.0,5.0,197.20,59.3
1,1,2016-11-04 22:12:00,2016-11-04 22:25:00,Business,Cory,Agnew,3.9,NO_DATA,22,4,...,41.6,0.0,0.0,0.0,0.0,0.0,6.2,13.3,257.92,61.1
2,2,2016-11-05 17:29:00,2016-11-05 17:40:00,Business,Renaissance,Agnew,2.8,NO_DATA,17,5,...,52.5,0.0,0.0,0.0,0.0,1.0,7.0,14.8,367.50,75.3
3,3,2016-11-06 16:05:00,2016-11-06 16:22:00,Business,Renaissance,Agnew,2.8,Meeting,16,6,...,43.9,1.0,0.0,0.0,0.0,0.0,16.3,34.6,715.57,94.8
4,4,2016-03-17 17:20:00,2016-03-17 18:02:00,Business,Midtown,Alief,15.5,Meal/Entertain,17,3,...,50.9,1.0,0.0,0.0,0.0,3.0,6.8,28.1,346.12,88.8


1. Let's do a little bit of data cleaning! CURRENTLY the "Top 5 Most Frequent Routes" are:

NO_DATA -> NO_DATA     86

Morrisville -> Cary    75

Cary -> Morrisville    67

Cary -> Cary           53

Cary -> Durham         36

We don't want [no route -> no route] produced, so let's exclude this from the output of "Top 5 Most Frequent Routes":
* Remember to format the output as (start) -> (stop)
* Don't directly change the dataframe

In [6]:
#business
df_business = df_business[df_business['START'] != 'No Data']
df_business = df_business[df_business['STOP'] != 'No Data']

2. Find the demand (number of rides per hour), and segment it by
* hour
* day_of_week
* is_rush_hour

In [14]:
# Calculate demand (number of rides) by grouping rows
df_hour = df_business.groupby('hour').size()
df_day_of_week = df_business.groupby('day_of_week').size()

# Note: 'IS_RUSH_HOUR' needs to be defined based on your logic (e.g., peak hours)
# For now, let's look at the first two:
print("Rides per Hour:\n", df_hour)
print("\nRides per Day of Week:\n", df_day_of_week)

Rides per Hour:
 hour
0     17
1      5
2      2
3      3
5      4
6      4
7     12
8     34
9     50
10    60
11    61
12    70
13    89
14    83
15    91
16    84
17    90
18    88
19    63
20    67
21    48
22    30
23    22
dtype: int64

Rides per Day of Week:
 day_of_week
0    165
1    162
2    138
3    147
4    191
5    132
6    142
dtype: int64


3. Using scipy's T-test function, compare rush vs non-rush demand by the hour.

In [15]:
# 1. Ensure START_DATE is in datetime format
df_business['START_DATE'] = pd.to_datetime(df_business['START_DATE'])

# Define rush hour logic (e.g., 7-9 AM and 4-6 PM)
def check_rush(hour):
    return 1 if hour in [7, 8, 9, 16, 17, 18] else 0

df_business['is_rush_hour'] = df_business['hour'].apply(check_rush)

# 2. Create a 'hour_bucket' column
df_business['hour_bucket'] = df_business['START_DATE'].dt.floor('h')

# 3. Aggregate data to get demand (rides count) per hour bucket
demand_df = df_business.groupby('hour_bucket').agg({'is_rush_hour': 'first', 'START': 'count'}).rename(columns={'START': 'rides_count'})

# 4. Separate the demand into Rush and Non-Rush groups
Rush = demand_df[demand_df['is_rush_hour'] == 1]['rides_count']
Non_Rush = demand_df[demand_df['is_rush_hour'] == 0]['rides_count']

# 5. Perform the T-test using stats (imported from scipy)
t_statistic, p_value = stats.ttest_ind(Rush, Non_Rush)

# 6. Display Results
print("Rush Hour Mean:", Rush.mean())
print("Non-Rush Hour Mean:", Non_Rush.mean())
print("p-value:", p_value)
print("t-statistic:", t_statistic)

# 7. Determine significance
if p_value < 0.05:
    print("There is a significant difference in demand between Rush and Non-Rush hours.")
else:
    print("There is no significant difference in demand.")

Rush Hour Mean: 1.1699346405228759
Non-Rush Hour Mean: 1.1147286821705427
p-value: 0.028460583058290932
t-statistic: 2.1942309538140297
There is a significant difference in demand between Rush and Non-Rush hours.


4. Day of week analysis to test whether average demand differs by day: ANOVA updates

In [30]:
# 1. Return a sub-dataframe: daily_hourly_demand
# Group by 'hour_bucket' to get rides per hour, keeping the day_of_week
daily_hourly_demand = df_business.groupby('hour_bucket').agg({
    'day_of_week': 'first',
    'START': 'count'
}).rename(columns={'START': 'rides_count'})

# 2. Prepare data: Create a list of groups of ride counts of each hour bucket per day
days = sorted(daily_hourly_demand['day_of_week'].unique())
groups = [daily_hourly_demand[daily_hourly_demand['day_of_week'] == day]['rides_count'] for day in days]

# 3. ANOVA
f_stat, p_val_anova = stats.f_oneway(*groups)

# 4. print whether p-value is significant or not
print(f"ANOVA F-statistic: {f_stat:.4f}")
print(f"ANOVA p-value: {p_val_anova:.4f}")

if p_val_anova < 0.05:
    print("There is a significant difference in average hourly demand across different days of the week.")
else:
    print("There is no significant difference in average hourly demand across different days of the week.")

ANOVA F-statistic: 1.3879
ANOVA p-value: 0.2164
There is no significant difference in average hourly demand across different days of the week.


Confidence intervals (answer the overarching question)

In [21]:
# pick a time and day and create a series
# Example: Friday (day 4) at 5 PM (hour 17)
sample_series = df_business[(df_business['day_of_week'] == 4) & (df_business['hour'] == 17)]

# We need the hourly demand counts for this specific slice across all weeks
# Grouping by the date (hour_bucket) ensures we get the count for each specific Friday at 5pm
demand_series = sample_series.groupby('hour_bucket').size()

# make mean and sem variables
mean_demand = demand_series.mean()
sem_demand = stats.sem(demand_series)

# make ci and print mean demand and ci
# Using 95% confidence level
confidence_level = 0.95
degrees_freedom = len(demand_series) - 1
ci = stats.t.interval(confidence_level, degrees_freedom, mean_demand, sem_demand)

print(f"Mean demand for Fridays at 5 PM: {mean_demand:.2f}")
print(f"95% Confidence Interval: ({ci[0]:.2f}, {ci[1]:.2f})")

Mean demand for Fridays at 5 PM: 1.00
95% Confidence Interval: (nan, nan)


/usr/local/lib/python3.12/dist-packages/scipy/stats/_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.12/dist-packages/scipy/stats/_distn_infrastructure.py:2324: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
